# AMLS final project

This notebook contains the work for the ELEC0134 Applied Machine Learning Systems class at UCL. The solution is implemented in Keras.

## Using Drive with Colab, typical imports

In [1]:
# Set up Google Drive for use with Colaboratory
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Magic calls TensorFlow 2.0 when importing
%tensorflow_version 2.x
import tensorflow as tf

# This tests whether a GPU is running
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Import Keras
from tensorflow import keras

# Check version of TensorFlow and Keras
print(tf.__version__)
print(keras.__version__)

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0
2.1.0-rc1
2.2.4-tf


## Changing directory, sanity checks

In [0]:
root_dir = "/content/drive/My Drive/"
change_dir = root_dir + "dataset_AMLS_19-20/celeba"

os.chdir(change_dir)

In [4]:
# Check current directory
!pwd

/content/drive/My Drive/dataset_AMLS_19-20/celeba


In [5]:
# List contents of directory
!ls

img  labels.csv


## Creating data input pipeline

In [0]:
# Import data as dataframe, drop unnecessary column
df = pd.read_csv("labels.csv", sep="\t", dtype=str)

# Create separate dataframes for gender and smiling
gender = df.copy()
smiling = df.copy()

gender.drop(gender.columns[0], axis=1, inplace=True)
gender.drop(gender.columns[2], axis=1, inplace=True)

smiling.drop(smiling.columns[0], axis=1, inplace=True)
smiling.drop(smiling.columns[1], axis=1, inplace=True)

In [7]:
# Inspect data
df.head()

,Unnamed: 0,img_name,gender,smiling
0,0,0.jpg,-1,1
1,1,1.jpg,-1,1
2,2,2.jpg,1,-1
3,3,3.jpg,-1,-1
4,4,4.jpg,-1,-1


In [8]:
gender.head()

,img_name,gender
0,0.jpg,-1
1,1.jpg,-1
2,2.jpg,1
3,3.jpg,-1
4,4.jpg,-1


In [9]:
len(gender)

5000

In [10]:
smiling.head()

,img_name,smiling
0,0.jpg,1
1,1.jpg,1
2,2.jpg,-1
3,3.jpg,-1
4,4.jpg,-1


In [0]:
# Now, we create a training and test sets for the gender and smiling datasets

from sklearn.model_selection import train_test_split

gender_train, gender_test = train_test_split(
    gender, 
    test_size=0.2,
    random_state=42
    )

smiling_train, smiling_test = train_test_split(
    smiling, 
    test_size=0.2, 
    random_state=42
    )

In [12]:
gender_train.head()

,img_name,gender
4227,4227.jpg,-1
4676,4676.jpg,1
800,800.jpg,1
3671,3671.jpg,-1
4193,4193.jpg,-1


In [13]:
len(gender_train)

4000

In [14]:
smiling_train.head()

,img_name,smiling
4227,4227.jpg,1
4676,4676.jpg,1
800,800.jpg,-1
3671,3671.jpg,1
4193,4193.jpg,-1


In [15]:
# We check the average dimensions of the images in the dataset

# Code from
# https://towardsdatascience.com/image-classification-python-keras-tutorial-kaggle-challenge-45a6332a58b8

from PIL import Image

def get_size_statistics(DIR, number_of_files):
  heights = []
  widths = []
  counter = 1
  for img in os.listdir(DIR):
    path = os.path.join(DIR, img)
    print("Opening " + path + ": " + str(counter) + "/" + str(number_of_files))
    data = np.array(Image.open(path)) # PIL Image library
    heights.append(data.shape[0])
    widths.append(data.shape[1])
    counter += 1
  avg_height = sum(heights) / len(heights)
  avg_width = sum(widths) / len(widths)
  print('\n')
  print("Average Height: " + str(avg_height))
  print("Max Height: " + str(max(heights)))
  print("Min Height: " + str(min(heights)))
  print('\n')
  print("Average Width: " + str(avg_width))
  print("Max Width: " + str(max(widths)))
  print("Min Width: " + str(min(widths)))

get_size_statistics("img/", len(df))

Opening img/1398.jpg: 1/5000
Opening img/4586.jpg: 2/5000
Opening img/2849.jpg: 3/5000
Opening img/142.jpg: 4/5000
Opening img/94.jpg: 5/5000
Opening img/618.jpg: 6/5000
Opening img/2685.jpg: 7/5000
Opening img/80.jpg: 8/5000
Opening img/4592.jpg: 9/5000
Opening img/1367.jpg: 10/5000
Opening img/3216.jpg: 11/5000
Opening img/195.jpg: 12/5000
Opening img/4579.jpg: 13/5000
Opening img/2108.jpg: 14/5000
Opening img/1401.jpg: 15/5000
Opening img/3564.jpg: 16/5000
Opening img/3570.jpg: 17/5000
Opening img/1415.jpg: 18/5000
Opening img/2652.jpg: 19/5000
Opening img/1373.jpg: 20/5000
Opening img/4545.jpg: 21/5000
Opening img/181.jpg: 22/5000
Opening img/5.jpg: 23/5000
Opening img/2134.jpg: 24/5000
Opening img/3202.jpg: 25/5000
Opening img/4223.jpg: 26/5000
Opening img/817.jpg: 27/5000
Opening img/803.jpg: 28/5000
Opening img/2646.jpg: 29/5000
Opening img/57.jpg: 30/5000
Opening img/43.jpg: 31/5000
Opening img/4551.jpg: 32/5000
Opening img/3558.jpg: 33/5000
Opening img/4237.jpg: 34/5000
Openin

In [16]:
# We now create two ImageDataGenerator objects for the gender dataset:
# one for training, the other for validation
from keras.preprocessing.image import ImageDataGenerator

# https://forums.fast.ai/t/split-data-using-fit-generator/4380/4
# for validation split

# We rescale to ensure RGB values fall between 0 and 1
# We set aside 20% of the training set for validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# We generate an image-label pair for the training set as follows
gender_train_gen = datagen.flow_from_dataframe(
    dataframe=gender_train, 
    directory="img/",
    x_col="img_name",
    y_col="gender",
    class_mode="binary",
    target_size=(218,178),
    batch_size=32,
    subset="training"
    )

# We generate an image-label pair for the validation set as follows
gender_val_gen = datagen.flow_from_dataframe(
    dataframe=gender_train,
    directory="img/",
    x_col="img_name",
    y_col="gender",
    class_mode="binary",
    target_size=(218,178),
    batch_size=32,
    subset="validation"
    )

Using TensorFlow backend.


Found 3200 validated image filenames belonging to 2 classes.
Found 800 validated image filenames belonging to 2 classes.


In [21]:
# Create two ImageDataGenerator objects for the smiling dataset in a similar way

# https://forums.fast.ai/t/split-data-using-fit-generator/4380/4
# for validation split

# We rescale to ensure RGB values fall between 0 and 1
# We set aside 20% of the training set for validation
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2)

# We generate an image-label pair for the training set as follows
smiling_train_gen = datagen.flow_from_dataframe(
    dataframe=smiling_train,
    directory="img/",
    x_col="img_name",
    y_col="smiling",
    class_mode="binary",
    target_size=(218,178),
    batch_size=32,
    subset="training"
    )

# We generate an image-label pair for the validation set as follows
smiling_val_gen = datagen.flow_from_dataframe(
    dataframe=smiling_train,
    directory="img/",
    x_col="img_name",
    y_col="smiling",
    class_mode="binary",
    target_size=(218,178),
    batch_size=32,
    subset="validation"
    )

Found 3200 validated image filenames belonging to 2 classes.
Found 800 validated image filenames belonging to 2 classes.


# Building and training the models



### Multi-layer Perceptron (MLP)

In [0]:
# Creating a classification MLP with two hidden layers
# We are using the Sequential API which creates a stack of layers
# in which the input flows through one after the other

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Instantiate Sequential API
mlp_model = keras.models.Sequential()

# This flattens the 218x178x3 input into a 1D tensor
mlp_model.add(keras.layers.Flatten(input_shape=(218,178,3)))

# This adds a fully connected layer with 300 neurons using the ReLU
# activation function
mlp_model.add(keras.layers.Dense(300, activation="relu"))

# This adds a fully connected layer with 100 neurons using the ReLU
# activation function
mlp_model.add(keras.layers.Dense(100, activation="relu"))

# This creates the output layer.
mlp_model.add(keras.layers.Dense(1, activation="sigmoid"))

In [0]:
# We now compile the MLP model to specify the loss function
# and the optimizer to use (SGD)
mlp_model.compile(loss="binary_crossentropy", # b/c of exclusive, sparse outputs
                  optimizer="sgd", # We use SGD to optimise the ANN
                  metrics=["accuracy"] # Used for classifiers
                  ) 

In [23]:
# Training and evaluating the MLP model on the gender dataset
mlp_gender_history = mlp_model.fit_generator(
    gender_train_gen,
    steps_per_epoch=gender_train_gen.samples // 32,
    validation_data=gender_val_gen,
    validation_steps=gender_val_gen.samples // 32,
    epochs=30
    )

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 25 steps
Epoch 1/30
100/100 [==============================] - 11s 111ms/step - loss: 0.8394 - accuracy: 0.5659 - val_loss: 0.7655 - val_accuracy: 0.4988
Epoch 2/30
100/100 [==============================] - 9s 92ms/step - loss: 0.6261 - accuracy: 0.6597 - val_loss: 0.6881 - val_accuracy: 0.6025
Epoch 3/30
100/100 [==============================] - 9s 93ms/step - loss: 0.5756 - accuracy: 0.7081 - val_loss: 0.6408 - val_accuracy: 0.6775
Epoch 4/30
100/100 [==============================] - 9s 90ms/step - loss: 0.5527 - accuracy: 0.7234 - val_loss: 0.6491 - val_accuracy: 0.6862
Epoch 5/30
100/100 [==============================] - 9s 89ms/step - loss: 0.5012 - accuracy: 0.7656 - val_loss: 0.5433 - val_accuracy: 0.7175
Epoch 6/30
100/100 [==============================] - 9s 90ms/step - loss: 0.4790 - accuracy: 0.7809 - val_loss: 0

In [24]:
# Training and evaluating the MLP model on the smiling dataset
mlp_smiling_history = mlp_model.fit_generator(
    smiling_train_gen,
    steps_per_epoch=smiling_train_gen.samples // 32,
    validation_data=smiling_validation_gen,
    validation_steps=smiling_validation_gen.samples // 32,
    epochs=30
    )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 25 steps
Epoch 1/30
100/100 [==============================] - 9s 87ms/step - loss: 0.7372 - accuracy: 0.5200 - val_loss: 0.6819 - val_accuracy: 0.5650
Epoch 2/30
100/100 [==============================] - 9s 86ms/step - loss: 0.6684 - accuracy: 0.5897 - val_loss: 0.6563 - val_accuracy: 0.6275
Epoch 3/30
100/100 [==============================] - 9s 88ms/step - loss: 0.6458 - accuracy: 0.6294 - val_loss: 0.6044 - val_accuracy: 0.6625
Epoch 4/30
100/100 [==============================] - 9s 87ms/step - loss: 0.6177 - accuracy: 0.6656 - val_loss: 0.6023 - val_accuracy: 0.6712
Epoch 5/30
100/100 [==============================] - 9s 90ms/step - loss: 0.5846 - accuracy: 0.6906 - val_loss: 0.7920 - val_accuracy: 0.5475
Epoch 6/30
100/100 [==============================] - 9s 88ms/step - loss: 0.5667 - accuracy: 0.7122 - val_loss: 0.6033 - val_accuracy: 0.6650
Epoch 7/30
100/100 [=============================

### Convolutional Neural Network (CNN)

In [0]:
# We create a simple CNN architecture for image classification
# Architecture from:
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb#scrollTo=L1WtoaOHVrVh

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

cnn_model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(218,178,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
    ])

In [0]:
# We now compile the CNN model to specify the loss function
# and the optimizer to use (Adam)
cnn_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

In [28]:
# Training and evaluating the CNN model on the gender dataset
cnn_gender_history = cnn_model.fit(
    gender_train_gen,
    steps_per_epoch=gender_train_gen.samples // 32,
    validation_data=gender_val_gen,
    validation_steps=gender_val_gen.samples // 32,
    epochs=30
    )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 25 steps
Epoch 1/30
100/100 [==============================] - 14s 138ms/step - loss: 0.6360 - accuracy: 0.6859 - val_loss: 0.4926 - val_accuracy: 0.7862
Epoch 2/30
100/100 [==============================] - 9s 88ms/step - loss: 0.3138 - accuracy: 0.8675 - val_loss: 0.3279 - val_accuracy: 0.8712
Epoch 3/30
100/100 [==============================] - 9s 88ms/step - loss: 0.1917 - accuracy: 0.9269 - val_loss: 0.2929 - val_accuracy: 0.9025
Epoch 4/30
100/100 [==============================] - 9s 87ms/step - loss: 0.1209 - accuracy: 0.9572 - val_loss: 0.3500 - val_accuracy: 0.8813
Epoch 5/30
100/100 [==============================] - 9s 89ms/step - loss: 0.0884 - accuracy: 0.9666 - val_loss: 0.2967 - val_accuracy: 0.8925
Epoch 6/30
100/100 [==============================] - 9s 88ms/step - loss: 0.0480 - accuracy: 0.9834 - val_loss: 0.3530 - val_accuracy: 0.8950
Epoch 7/30
100/100 [===========================

In [29]:
# Training and evaluating the CNN model on the smiling dataset
cnn_smiling_history = cnn_model.fit(
    smiling_train_gen,
    steps_per_epoch=smiling_train_gen.samples // 32,
    validation_data=smiling_validation_gen,
    validation_steps=smiling_validation_gen.samples // 32,
    epochs=30
    )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 25 steps
Epoch 1/30
100/100 [==============================] - 9s 86ms/step - loss: 0.8483 - accuracy: 0.5650 - val_loss: 0.5900 - val_accuracy: 0.7075
Epoch 2/30
100/100 [==============================] - 9s 88ms/step - loss: 0.4711 - accuracy: 0.7794 - val_loss: 0.4212 - val_accuracy: 0.8012
Epoch 3/30
100/100 [==============================] - 9s 86ms/step - loss: 0.3793 - accuracy: 0.8309 - val_loss: 0.3929 - val_accuracy: 0.8175
Epoch 4/30
100/100 [==============================] - 9s 86ms/step - loss: 0.3290 - accuracy: 0.8606 - val_loss: 0.3789 - val_accuracy: 0.8363
Epoch 5/30
100/100 [==============================] - 9s 87ms/step - loss: 0.2953 - accuracy: 0.8684 - val_loss: 0.5087 - val_accuracy: 0.8062
Epoch 6/30
100/100 [==============================] - 9s 86ms/step - loss: 0.2588 - accuracy: 0.8922 - val_loss: 0.4198 - val_accuracy: 0.8288
Epoch 7/30
100/100 [=============================

## Next Steps

* Figure out the principles for building a good MLP and CNN and  architecture for image classification
* Read chapter on training deep neural nets
* Rewatch fast.ai lecture 1 on finetuning
* Learn how to implement a model interpreter (similar to that of fast.ai)
* Find out how to implement more advanced CNN architectures for image classification
* Do literature review
* Write the code in a format that works for submission